In [1]:
import os

In [2]:
%pwd

'c:\\AJM\\Coding Languages\\Text_Summarization\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\AJM\\Coding Languages\\Text_Summarization\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_dir

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_dir([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config["root_dir"]),
            data_path=Path(config["data_path"]),
            model_ckpt=config["model_ckpt"],
            num_train_epochs=params["num_train_epochs"],
            warmup_steps=params["warmup_steps"],
            per_device_train_batch_size=params["per_device_train_batch_size"],
            weight_decay=params["weight_decay"],
            logging_steps=params["logging_steps"],
            evaluation_strategy=params["evaluation_strategy"],
            eval_steps=params["eval_steps"],
            save_steps=params["save_steps"],
            gradient_accumulation_steps=params["gradient_accumulation_steps"]
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2025-04-10 13:56:01,892: WARNING: module_wrapper - From c:\Users\Agnya Mistry\.conda\envs\textS\lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2025-04-10 13:56:03,780: INFO: config - PyTorch version 2.6.0 available.]
[2025-04-10 13:56:03,792: INFO: config - TensorFlow version 2.19.0 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        import os

        if not os.path.exists(self.config.data_path):
            raise FileNotFoundError(f"Dataset path does not exist: {self.config.data_path}")
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=int(1e6),
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-10 13:56:14,287: INFO: common - yamle file: config\config.yaml read successfully]
[2025-04-10 13:56:14,312: INFO: common - yamle file: params.yaml read successfully]
[2025-04-10 13:56:14,319: INFO: common - Directory already exists: artifacts]
[2025-04-10 13:56:14,324: INFO: common - Directory already exists: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Agnya Mistry\AppData\Local\Temp\ipykernel_150440\1387368500.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


Step,Training Loss,Validation Loss


c:\Users\Agnya Mistry\.conda\envs\textS\lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [23]:
import transformers
print(transformers.__version__)

4.51.1
